In [ ]:
from symbolic_hulls_func_aggr import *

# Geometry
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Data handling and combinations
from itertools import cycle
from collections import defaultdict

# Plotly imports
import plotly.graph_objects as go
from plotly.colors import sequential

# Matplotlib for additional plotting
import matplotlib.pyplot as plt

## Functions

In [ ]:
def plot_phases(phases):
    fig = go.Figure()

    # Record the index of the phase and the number of phases
    phases = [(i, phase) for i, phase in enumerate(phases)]

    # Define the range for x and y and create a mesh
    x_vals = np.linspace(-7, 7, 300)
    y_vals = np.linspace(-7, 7, 300)
    x_mesh, y_mesh = np.meshgrid(x_vals, y_vals)

    # Define the color scales
    colorscales = [
        'Viridis',
        'emrld',
        'Inferno',
        'Cividis',
        'Turbo',
        'Bluered'
    ]

    phase_diagram_colorscale = sequential.Electric

    # Plot all the energy surfaces
    for phase in phases:
        phase_func = phase[1]
        i = phase[0]

        z_vals = sp.lambdify((x, y), phase_func, 'numpy')(x_mesh, y_mesh)
        fig.add_trace(go.Surface(z=z_vals, x=x_mesh, y=y_mesh, colorscale=colorscales[i], showscale=False))

    fig.update_layout(showlegend=False,
                    width=600,
                    height=600, 
                    scene=dict(
                        xaxis_title='X',
                        yaxis_title='Y',
                        zaxis_title='Z',
                        zaxis=dict(range=[-1, 20])),  # Adjust the range as needed
                        coloraxis_showscale=False,
                    title='3D Phases with 2D Coexistance')

    fig.show()

In [ ]:
def plot_phase_diagram(points, labels, mode='2d'):
    '''This will plot the phase diagram with a convex hull'''

    # Calculate the convex hull
    simplices = lower_convex_hull(points)

    # colors to cycle through
    color_cycle = cycle(['blue', 'red', 'yellow', 'green', 'purple', 'cyan', 'orange', 'pink', 'brown', 'gray'])
    
    if mode == '2d':
        colored_faces = defaultdict(list)
    elif mode == '3d':
        colored_faces = defaultdict(lambda: {'x': [], 'y': [], 'z': [], 'i': [], 'j': [], 'k': []})

    label_color_map = {}
    for simplex in simplices:
        unique_labels = frozenset(labels[simplex])  # Using frozenset to make it hashable

        # Check if we already have a color for this unique label set
        if unique_labels not in label_color_map:
            # Assign a new color from the cycle
            label_color_map[unique_labels] = next(color_cycle)

        color = label_color_map[unique_labels]
        
        # Extract the coordinates
        x_coords = points[simplex, 0]
        y_coords = points[simplex, 1]

        if mode == '2d':
            # Create a 2D polygon and add it to the color group for 2D projection
            polygon = Polygon(zip(x_coords, y_coords))  # Projected onto xy-plane
            colored_faces[color].append(polygon)

        elif mode == '3d':
            z_coords = points[simplex, 2]
            
            # Append the vertices to the respective color group
            index_offset = len(colored_faces[color]['x'])  # Offset for current group’s indices
            colored_faces[color]['x'].extend(x_coords)
            colored_faces[color]['y'].extend(y_coords)
            colored_faces[color]['z'].extend(z_coords)
            colored_faces[color]['i'].append(index_offset)
            colored_faces[color]['j'].append(index_offset + 1)
            colored_faces[color]['k'].append(index_offset + 2)

    fig = go.Figure()

    if mode == '2d':
        # Combine polygons that share edges for each color
        for color in colored_faces:
            # Combine the list of polygons into a single geometry
            combined_polygon = unary_union(colored_faces[color])
            # Replace the list with the combined polygon
            colored_faces[color] = combined_polygon

        for color, geometry in colored_faces.items():
            # Handle both Polygon and MultiPolygon geometries
            if geometry.geom_type == 'Polygon':
                geometries = [geometry]
            elif geometry.geom_type == 'MultiPolygon':
                geometries = geometry.geoms
            else:
                continue  # Skip if not a polygonal geometry

            for poly in geometries:
                x, y = list(poly.exterior.xy[0]), list(poly.exterior.xy[1])
                fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    fill="toself",
                    mode="none",
                    fillcolor=color,
                    opacity=0.5,
                    name=f"2D Projection - {color}"
                ))

                # Plot interior rings (holes) if any
                for interior in poly.interiors:
                    x_int, y_int = interior.xy
                    fig.add_trace(go.Scatter(
                        x=x_int,
                        y=y_int,
                        fill="toself",
                        mode="none",
                        fillcolor='white',
                        opacity=1,
                        showlegend=False
                    ))

        # Update layout
        fig.update_layout(
            scene=dict(
                xaxis=dict(title='X-axis', range=[-6, 6]),
                yaxis=dict(title='Y-axis', range=[-7, 5]),
            ),
            title='Phase Diagram in 2D',
            width=600,
            height=600,
            showlegend=False
        )
        
    elif mode =='3d':
        for color, data in colored_faces.items():
            fig.add_trace(go.Mesh3d(
                            x=data['x'],
                            y=data['y'],
                            z=data['z'],
                            i=data['i'],
                            j=data['j'],
                            k=data['k'],
                            color=color,
                            opacity=0.5,
                        ))

        # Update layout
        fig.update_layout(
            scene=dict(
                xaxis=dict(title='X-axis', range=[-6, 6]),
                yaxis=dict(title='Y-axis', range=[-7, 5]),
                zaxis=dict(title='Z-axis', range=[-6, 10]),
            ),
            title='Phase Diagram in 3D',
            width=600,
            height=600,
        )

    fig.show()

In [ ]:
def plot_phase_boundaries():
    fig = go.Figure()

    for i in range(len(yvals)):
        fig.add_trace(go.Scatter3d(
            x=xvals,
            y=yvals[i],
            z=zvals[i],
            mode='lines',
            line=dict(color='black', width=5)
        )
    )

    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X-axis', range=[-6, 6]),
            yaxis=dict(title='Y-axis', range=[-6, 6]),
            zaxis=dict(title='Z-axis', range=[-1, 10]),
        ),
        title='Phase Coexistence Boundaries',
        width=600,
        height=600,
        showlegend=False
    )

    fig.show()

In [ ]:
def plot_hull_on_phase_boundaries():
    # Compute the convex hull
    simplices = lower_convex_hull(points)

    fig = go.Figure()

    # Extract the vertices of each triangle in the hull
    fig.add_trace(go.Mesh3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        i=simplices[:, 0],  # First vertex of each triangle
        j=simplices[:, 1],  # Second vertex of each triangle
        k=simplices[:, 2],  # Third vertex of each triangle
        color='lightblue',
        opacity=0.5
    ))

    for i in range(len(yvals)):
        fig.add_trace(go.Scatter3d(
            x=xvals,
            y=yvals[i],
            z=zvals[i],
            mode='lines',
            line=dict(color='black', width=5)
        )
    )
        
    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X-axis', range=[-6, 6]),
            yaxis=dict(title='Y-axis', range=[-7, 5]),
            zaxis=dict(title='Z-axis', range=[-1, 10]),
        ),
        title='Phase Diagram',
        width=600,
        height=600,
        showlegend=False
    )

    # Show the figure
    fig.show()

## Example Functions and Calculation

In [ ]:
# define your funcitons here in explicit form
x, y, z = sp.symbols('x y z')

# define your funcitons here in explicit form
f1 = x**2 + y**2
f2 = (x-1)**2 + (y+4)**2 + 1
f3 = (x+3)**2 + (y+2)**2 + 1

phases = [f1, f2, f3]

In [ ]:
f1f2 = sp.solve(hpboundry_f2_to_f1(f1, f2), y)[0]
f2f1 = sp.solve(hpboundry_f2_to_f1(f2, f1), y)[0]
f1f3 = sp.solve(hpboundry_f2_to_f1(f1, f3), y)[0]
f3f1 = sp.solve(hpboundry_f2_to_f1(f3, f1), y)[0]
f2f3 = sp.solve(hpboundry_f2_to_f1(f2, f3), y)[0]
f3f2 = sp.solve(hpboundry_f2_to_f1(f3, f2), y)[0]

In [ ]:
xvals = np.arange(-6, 6, .1)

f1f2_yvals = sp.lambdify(x, f1f2, 'numpy')(xvals)
f2f1_yvals = sp.lambdify(x, f2f1, 'numpy')(xvals)
f1f3_yvals = sp.lambdify(x, f1f3, 'numpy')(xvals)
f3f1_yvals = sp.lambdify(x, f3f1, 'numpy')(xvals)
f2f3_yvals = sp.lambdify(x, f2f3, 'numpy')(xvals)
f3f2_yvals = sp.lambdify(x, f3f2, 'numpy')(xvals)

yvals = [f1f2_yvals, f2f1_yvals, f1f3_yvals, f3f1_yvals, f2f3_yvals, f3f2_yvals]

f1f2_zvals = sp.lambdify((x, y), f1, 'numpy')(xvals, f1f2_yvals)
f2f1_zvals = sp.lambdify((x, y), f2, 'numpy')(xvals, f2f1_yvals)
f1f3_zvals = sp.lambdify((x, y), f1, 'numpy')(xvals, f1f3_yvals)
f3f1_zvals = sp.lambdify((x, y), f3, 'numpy')(xvals, f3f1_yvals)
f2f3_zvals = sp.lambdify((x, y), f2, 'numpy')(xvals, f2f3_yvals)
f3f2_zvals = sp.lambdify((x, y), f3, 'numpy')(xvals, f3f2_yvals)

zvals = [f1f2_zvals, f2f1_zvals, f1f3_zvals, f3f1_zvals, f2f3_zvals, f3f2_zvals]

In [ ]:
points = []
labels = []

for xi, yi, zi in zip(xvals, f1f2_yvals, f1f2_zvals):
    points.append([xi, yi, zi])
    labels.append('1')

for xi, yi, zi in zip(xvals, f2f1_yvals, f2f1_zvals):
    points.append([xi, yi, zi])
    labels.append('2')

for xi, yi, zi in zip(xvals, f1f3_yvals, f1f3_zvals):
    points.append([xi, yi, zi])
    labels.append('1')

for xi, yi, zi in zip(xvals, f3f1_yvals, f3f1_zvals):
    points.append([xi, yi, zi])
    labels.append('3')

for xi, yi, zi in zip(xvals, f2f3_yvals, f2f3_zvals):
    points.append([xi, yi, zi])
    labels.append('2')

for xi, yi, zi in zip(xvals, f3f2_yvals, f3f2_zvals):
    points.append([xi, yi, zi])
    labels.append('3')

points = np.array(points)
labels = np.array(labels)

In [ ]:
point_mask = points[:, 2] <= 12

points = points[point_mask]
labels = labels[point_mask]

## Plotting

In [ ]:
f1, f2, f3 = sp.Symbol('f1'), sp.Symbol('f2'), sp.Symbol('f3')
display(sp.Eq(f1, phases[0]))
display(sp.Eq(f2, phases[1]))
display(sp.Eq(f3, phases[2]))

In [ ]:
# We can plot the parabolas to get a sense of the single phase energy surfaces
plot_phases(phases)

In [ ]:
plot_phase_boundaries()

In [ ]:
plot_hull_on_phase_boundaries()

In [ ]:
plot_phase_diagram(points, labels, mode='2d')

In [ ]:
# plot_phase_diagram(points, labels, mode='3d')